In [60]:
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup,NavigableString, Tag
import time
import requests
import re
import csv
from selenium.webdriver.common.by import By
import pandas as pd
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import os

driver = webdriver.Chrome()
    
driver.get("https://img.pcstore.com.tw/store_ad/bank/pop.htm#") #自動打開網頁（不要關）
    
time.sleep(5) #暫停
    
soup = BeautifulSoup(driver.page_source, 'html.parser') #把script變成美麗湯可讀
#print(soup.prettify())  #輸出排版後的HTML內容

bank_name =[] #存銀行名
results = soup.find_all("button", class_='BKbutton')
for result in results:
    name = result.find('img').get('alt')
    bank_name.append(name)
    #print(name)
    
#for j in range(len(bank_name)):
    #print(bank_name[j])
    
    
results = soup.find_all('div', class_='lic')
k = 0
for result in results:
    discount_time =[] #活動期間
    time = [] #計算有幾個活動時間
    discount =[] #活動
    title = [] #計算活動標題
    notice = [] #注意事項
    notice_num = [] #計算注意事項數量
    text = result.find('h1').text.split('\n')
    text_ = []
    for i  in range(len(text)):
        if ('活動時間' in text[i] and '/' in text[i]) or ('活動期間' in text[i] and '/' in text[i]):
            discount_time.append(text[i])
            time.append('活動期間')
        text_.append(text[i])
        #print(text[i])
        if i + 1 < len(text):
            if text[i+1] == '' or text[i+1] == None:
                text1 = '\n'.join(text_)
                discount.append(text1)
                #title.append('活動內容')
                #print(text1)
                text_= []
        else:
            text1= '\n'.join(text_)
            discount.append(text1)
            #title.append('活動內容')
            text_= []
    result2 = result.find_all('li')      
    for tex in result2:
        notice.append(tex.text)
        #print(tex.text)
        notice_num.append('注意事項')
    #print(discount_time)
    #print(discount)
    #print('$')
    discount = list(filter(lambda x: x != "", discount)) #刪除空字串
    for i in range(len(discount)):
        title.append('活動內容')
    bank_dict = {
        '標題':time+title+notice_num,
        '內容':discount_time+discount+notice, 
    }

    df = pd.DataFrame(bank_dict)
    #創資料夾
        
    folder_name = 'pchome商店街csv'
    path = "/Users/takoyaki/Downloads/購物平台爬蟲"  ##要改路徑
    
    folder_path = os.path.join(path, folder_name)
    if not os.path.isdir(folder_path):
        os.makedirs(folder_path, mode=0o777)
    path0 = str(path+'/'+folder_name+"/")
    #print(path0)
    df.to_csv(path0+('pchome商店街_'+bank_name[k]+'.csv'), index =  False)
    k+=1